In [1]:
import torch as t
import torch.nn as nn
from torch.autograd import Variable as V

input = V(t.randn(2, 3))
model = nn.Linear(3, 4)
output1 = model(input)  # 得到输出方式1
output2 = nn.functional.linear(input, model.weight, model.bias)  # 得到输出方式2
print(output1 == output2)

tensor([[True, True, True, True],
        [True, True, True, True]])


In [2]:
import torch as t
import torch.nn as nn

input = t.randn(2, 3)
model = nn.Linear(3, 4)
output1 = model(input)  # 得到输出方式1
output2 = nn.functional.linear(input,
                               model.weight, model.bias)  # 得到输出方式2
print(output1 == output2)
print(output1.shape)

tensor([[True, True, True, True],
        [True, True, True, True]])
torch.Size([2, 4])


In [3]:
from torch.utils.data import Dataset
import torch
import numpy as np


class MyDataset(Dataset):
    # Initialize your data, download, etc.
    def __init__(self):  # 读取csv文件中的数据
        xy = np.loadtxt('data-diabetes.csv', delimiter=',', dtype=np.float32)
        self.len = xy.shape[0]  # 除去最后一列为数据位，存在x_data中
        self.x_data = torch.from_numpy(xy[:, 0:-1])  # 最后一列为标签为，存y_data中
        self.y_data = torch.from_numpy(xy[:, [-1]])

    def __getitem__(self, index):  # 根据索引返回数据和对应的标签
        return self.x_data[index], self.y_data[index]

    def __len__(self):  # 返回文件数据的数目
        return self.len

In [4]:
from torch.utils.data import DataLoader
from torch.autograd import Variable

# 调用自己创建的Dataset
dataset = MyDataset()
train_loader = DataLoader(dataset=dataset, batch_size=32, shuffle=True, num_workers=2)
#循环来迭代来高效地获取数据
for i, data in enumerate(train_loader, 0):
    # get the inputs
    inputs, labels = data
    # wrap them in Variable
    inputs, labels = Variable(inputs), Variable(labels)
    # Run your training process
    print("step", i, "inputs", inputs.data, "labels", labels.data)

FileNotFoundError: data-diabetes.csv not found.

In [8]:
# 导入了resnet50的预训练模型
import torchvision
from multiprocessing import cpu_count

model = torchvision.models.resnet50(pretrained=True)
# 如果只需要网络结构，不需要用预训练模型的参数来初始化
# model = torchvision.models.resnet50(pretrained=False)
imagenet_data = torchvision.datasets.ImageNet('~/dataFolder/imagenet')
# data_loader = torch.utils.data.DataLoader(imagenet_data, batch_size=4, shuffle=True, num_workers=args.nThreads)
data_loader = torch.utils.data.DataLoader(imagenet_data, batch_size=4, shuffle=True, num_workers=cpu_count)
torchvision.datasets.MNIST('~/dataFolder/imagenet', train=True, transform=None, target_transform=None, download=False)

In [9]:
# 解决实际问题
# y = w * x^2 + bias
"""
使用深度学习解决问题的一般步骤：
（1）定义模型与确定参数
（2）准备数据
（3）确定损失函数与优化器
（4）训练并更新参数
"""

In [2]:
import torch
class MyLayer(torch.nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(MyLayer, self).__init__()  # 和自定义模型一样，第一句话就是调用父类的构造函数
        self.in_features = in_features
        self.out_features = out_features
        self.weight = torch.nn.Parameter(torch.Tensor(in_features, out_features))  # 由于weights是可以训练的，所以使用Parameter来定义。
        if bias:
            self.bias = torch.nn.Parameter(torch.Tensor(in_features))  # 由于bias是可以训练的，所以使用Parameter来定义
        else:
            self.register_parameter('bias', None)

        def forward(self, input):
            input_ = torch.pow(input, 2) + self.bias
            y = torch.matmul(input_, self.weight)
            return y

In [12]:
import torch
N, D_in, D_out = 10, 5, 3 # 一共10组样本，输入特征为5，输出特征为3
# 先定义一个模型
class MyNet(torch.nn.Module):
    def __init__(self):
        super(MyNet, self).__init__() # 第一句话，调用父类的构造函数
        self.mylayer1 = MyLayer(D_in,D_out)
        def forward(self, x):
            x = self.mylayer1(x)
            return x

In [13]:
model = MyNet()
print(model)

MyNet(
  (mylayer1): MyLayer()
)


In [14]:
# 创建输入、输出数据
x = torch.randn(N, D_in) #（10，5）
y = torch.randn(N, D_out) #（10，3）

In [15]:
loss_fn = torch.nn.MSELoss(reduction='sum') #定义损失函数
learning_rate = 1e-4 #定义学习率
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) #构造一个optimizer对象

In [16]:
for t in range(10):
    # 第一步：数据的前向传播，计算预测值p_pred
    y_pred = model(x)
    # 第二步：计算计算预测值p_pred与真实值的误差
    loss = loss_fn(y_pred, y)
    print(f"第 {t} 个epoch, 损失是 {loss.item()}")
    # 在反向传播之前，将模型的梯度归零
    optimizer.zero_grad()
    # 第三步：反向传播误差
    loss.backward()
    # 直接通过梯度一步到位，更新完整个网络的训练参数
    optimizer.step()

NotImplementedError: 